In [1]:
import sys, os, json, copy, sklearn, shap, gc, joblib, copy, time, ast
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle as pkl
from tqdm import tqdm
from smart_open import open
import seaborn as sns
from sklearn.metrics import roc_curve,roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from rdsutils.woe import WOE_Transform                                            
from rdsutils.metrics import get_pred_reports


sys.path.insert(1, "./")
from utility.performance_eval_plot import score_gain, plot_efficient_frontier, plot_percentile_bad_rate
import utility.performance_eval_v3 as p_eval
from utility  import data_summary, woe, data_eda, psi
from utility.governance import get_risk_plot,get_feature_by_importance, get_pdp, get_shap_dependence, pickle_obj, save_shap_dependence,\
    save_pdp, plot_hist, get_woe_plots

pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)

%load_ext autoreload
%autoreload 2

print(lgb.__version__)

3.2.1


### read data

In [2]:
config_path = "s3://sofi-data-science/hpeng/pl-gen4/artifact/config.json"
artifact_path = "s3://sofi-data-science/hpeng/pl-gen4/artifact"
with open(os.path.join(config_path), "r") as f:
    config = json.load(f)

In [3]:
exp_dict = pd.read_csv(config["meta"]["exp_dict_path"])
af_dict = pd.read_csv(config["meta"]["af_member_cust_dict_path"])

In [4]:
model_member = pkl.load(open('artifact/'+'member'+'_final_model.pkl', 'rb'))
model_non_member = pkl.load(open('artifact/'+'non_member'+'_final_model.pkl', 'rb'))
model_non_member29 = pkl.load(open('artifact/'+'non_member'+'_final_model_29.pkl', 'rb'))
model_non_member30 = pkl.load(open('artifact/'+'non_member'+'_final_model_30.pkl', 'rb'))
model_prescreen = pkl.load(open('artifact/'+'prescreen'+'_final_model.pkl', 'rb'))

In [5]:
feature_list = model_prescreen.feature_name_ + model_member.feature_name_ + model_non_member.feature_name_ \
            + model_non_member30.feature_name_ + config["data_columns"]["weights"] \
            + config["data_columns"]["target_cols"] + config["data_columns"]["benchmark_cols"] + \
            config["data_columns"]["meta_cols"] + ['all_time_lending_member_flag'] + ['funds_use_encode_v2','income_update']
feature_list = pd.unique(feature_list).tolist()

In [6]:
df_oot1_full = pd.read_parquet(config["data"]["clean"]["all_features_oot1"], columns = feature_list)
df_oot2_full = pd.read_parquet(config["data"]["clean"]["all_features_oot2"], columns = feature_list)
df_oot_full = pd.concat([df_oot1_full, df_oot2_full], axis=0, ignore_index=True)

df_oot_full['seg'] = 'other'
df_oot_full.loc[(df_oot_full.all_time_lending_member_flag==1), 'seg'] = 'member'
df_oot_full.loc[(df_oot_full.all_time_lending_member_flag!=1), 'seg'] = 'non_member'

### data preprecessing

In [7]:
def categorical_to_nan(series, categorical_list):
    """ given series and a list of catergorical values
    
    replace the categorical occurances to nan
    """
    if len(categorical_list) == 0:
        return series
    mapper = dict((k, np.nan) for k in categorical_list)
    return series.replace(mapper)

no_special_list = []

for col in tqdm(feature_list):
    try:
        special_val = ast.literal_eval(exp_dict[exp_dict.field_name == col].categorical.iloc[0])
        special_val = [int(i) for i in special_val]
        df_oot_full[col] = categorical_to_nan(df_oot_full[col], special_val)
    except:
        no_special_list.append(col)
        
len(no_special_list)

100%|██████████| 89/89 [00:09<00:00,  9.47it/s]


30

In [8]:
df_oot_full_member_ = df_oot_full[df_oot_full.seg=='member'].copy()
df_oot_full_non_member_ = df_oot_full[df_oot_full.seg=='non_member'].copy()
df_oot_full_prescreen_ = df_oot_full.copy()
df_oot_full_non_member_30_ = df_oot_full[df_oot_full.seg=='non_member'].copy()


impute_vals_member = pkl.load(open('artifact/'+'member'+'_final_imputed_val.pkl', 'rb'))
impute_vals_non_member = pkl.load(open('artifact/'+'non_member'+'_final_imputed_val.pkl', 'rb'))
impute_vals_prescreen = pkl.load(open('artifact/'+'prescreen'+'_final_imputed_val.pkl', 'rb'))
impute_vals_non_member_30 = pkl.load(open('artifact/'+'non_member'+'_final_imputed_val_30.pkl', 'rb'))


for f in tqdm(model_member.feature_name_):
    try:
        val = impute_vals_member[f]
        df_oot_full_member_[f].fillna(val, inplace=True)
    except:
        print("no special value for " + f)    
        
for f in tqdm(model_non_member.feature_name_):
    try:
        val = impute_vals_non_member[f]
        df_oot_full_non_member_[f].fillna(val, inplace=True)
    except:
        print("no special value for " + f)
        
for f in tqdm(model_prescreen.feature_name_):
    try:
        val = impute_vals_prescreen[f]
        df_oot_full_prescreen_[f].fillna(val, inplace=True)
    except:
        print("no special value for " + f)
        
for f in tqdm(model_non_member30.feature_name_):
    try:
        val = impute_vals_non_member_30[f]
        df_oot_full_non_member_30_[f].fillna(val, inplace=True)
    except:
        print("no special value for " + f)

100%|██████████| 31/31 [00:00<00:00, 1836.23it/s]


no special value for p13_iqz9420
no special value for p13_iqz9426
no special value for sofi_num_inq_12month


100%|██████████| 31/31 [00:00<00:00, 402.96it/s]


no special value for p13_iqz9427
no special value for sofi_num_inq_1month
no special value for sofi_num_inq_3month
no special value for sofi_num_inq_personal_finance


100%|██████████| 31/31 [00:00<00:00, 318.70it/s]


no special value for p13_iqz9420
no special value for p13_iqz9425


  0%|          | 0/30 [00:00<?, ?it/s]

no special value for p13_iqz9427
no special value for sofi_num_inq_1month
no special value for sofi_num_inq_3month
no special value for sofi_num_inq_personal_finance


100%|██████████| 30/30 [00:00<00:00, 415.95it/s]


### scoring

In [9]:
df_oot_full_member_['gen4_underwriting_prob'] = model_member.predict_proba(df_oot_full_member_[model_member.feature_name_])[:,1]
df_oot_full_non_member_['gen4_underwriting_prob'] = model_non_member.predict_proba(df_oot_full_non_member_[model_non_member.feature_name_])[:,1]

df_oot_full_member_['gen4_underwriting_prob_29'] = model_member.predict_proba(df_oot_full_member_[model_member.feature_name_])[:,1]
df_oot_full_non_member_['gen4_underwriting_prob_29'] = model_non_member29.predict_proba(df_oot_full_non_member_[model_non_member29.feature_name_])[:,1]

df_oot_full_member_['gen4_underwriting_prob_30'] = model_member.predict_proba(df_oot_full_member_[model_member.feature_name_])[:,1]
df_oot_full_non_member_30_['gen4_underwriting_prob_30'] = model_non_member30.predict_proba(df_oot_full_non_member_30_[model_non_member30.feature_name_])[:,1]

df_oot_ = pd.concat([df_oot_full_member_, df_oot_full_non_member_], axis=0, ignore_index=True)
df_oot_["weight_eval"] = df_oot_["weight_cob"] * df_oot_["weight_ri_v2"]

df_oot_30_ = pd.concat([df_oot_full_member_, df_oot_full_non_member_30_], axis=0, ignore_index=True)
df_oot_30_["weight_eval"] = df_oot_30_["weight_cob"] * df_oot_30_["weight_ri_v2"]

df_oot_full_prescreen_['gen4_prescreen_prob'] = model_prescreen.predict_proba(df_oot_full_prescreen_[model_prescreen.feature_name_])[:,1]
df_oot_full_prescreen_["weight_eval"] = df_oot_full_prescreen_["weight_cob"] * df_oot_full_prescreen_["weight_ri_v2"]

In [10]:
df_oot_.shape, df_oot_full_prescreen_.shape, df_oot_30_.shape

((1150725, 94), (1150725, 92), (1150725, 94))

In [11]:
df_oot_ = pd.merge(df_oot_[['id','applicant_type','date_start','seg','ri_source','target_v2','weight_eval',\
             'period',  'gen3_score','applicant_fico_score','applicant_vantage_score','funds_use_encode_v2','income_update',\
                            'gen4_underwriting_prob','gen4_underwriting_prob_29']], df_oot_full_prescreen_[['gen4_prescreen_prob','id','applicant_type','date_start','weight_eval']],
              how = 'inner', on = ['id', 'applicant_type', 'date_start','weight_eval'])
df_oot_ = pd.merge(df_oot_, df_oot_30_[['gen4_underwriting_prob_30','id','applicant_type','date_start','weight_eval']],
              how = 'inner', on = ['id', 'applicant_type', 'date_start','weight_eval'])

In [12]:
df_oot_.shape

(1150725, 17)

In [13]:
df_oot_.head()

,id,applicant_type,date_start,seg,ri_source,target_v2,weight_eval,period,gen3_score,applicant_fico_score,applicant_vantage_score,funds_use_encode_v2,income_update,gen4_underwriting_prob,gen4_underwriting_prob_29,gen4_prescreen_prob,gen4_underwriting_prob_30
0,6038637,PRIMARY,2018-03-19,member,others,0.0,0.997245,oot18q1,743.0,785.0,742.0,2,70000.0,0.039774,0.039774,0.063039,0.039774
1,6038637,PRIMARY,2018-03-19,member,others,1.0,0.002755,oot18q1,743.0,785.0,742.0,2,70000.0,0.039774,0.039774,0.063039,0.039774
2,5961856,PRIMARY,2018-03-07,member,proxy,1.0,1.000000,oot18q1,655.0,689.0,672.0,5,1466909.0,0.098705,0.098705,0.123878,0.098705
3,6078544,PRIMARY,2018-03-24,member,others,0.0,0.999152,oot18q1,698.0,744.0,649.0,1,85000.0,0.131640,0.131640,0.060741,0.131640
4,6078544,PRIMARY,2018-03-24,member,others,1.0,0.000848,oot18q1,698.0,744.0,649.0,1,85000.0,0.131640,0.131640,0.060741,0.131640


### probability to score transformation
- range 300~850
- 40 double odds
- odds 5:1 at score 650

In [14]:
factor = 40/np.log(2)
offset = 650 - factor * np.log(5)

df_oot_['uw_log_odds'] = np.log((1 - df_oot_['gen4_underwriting_prob'])/df_oot_['gen4_underwriting_prob'])
df_oot_['uw_odds'] = ((1 - df_oot_['gen4_underwriting_prob'])/df_oot_['gen4_underwriting_prob'])

df_oot_['gen4_underwriting_score'] = round(offset+factor*df_oot_['uw_log_odds'])
df_oot_.loc[df_oot_.gen4_underwriting_score>850,'gen4_underwriting_score']=850
df_oot_.loc[df_oot_.gen4_underwriting_score<300,'gen4_underwriting_score']=300

df_oot_['uw_log_odds_29'] = np.log((1 - df_oot_['gen4_underwriting_prob_29'])/df_oot_['gen4_underwriting_prob_29'])
df_oot_['uw_odds_29'] = ((1 - df_oot_['gen4_underwriting_prob_29'])/df_oot_['gen4_underwriting_prob_29'])

df_oot_['gen4_underwriting_score_29'] = round(offset+factor*df_oot_['uw_log_odds_29'])
df_oot_.loc[df_oot_.gen4_underwriting_score_29>850,'gen4_underwriting_score_29']=850
df_oot_.loc[df_oot_.gen4_underwriting_score_29<300,'gen4_underwriting_score_29']=300

df_oot_['uw_log_odds_30'] = np.log((1 - df_oot_['gen4_underwriting_prob_30'])/df_oot_['gen4_underwriting_prob_30'])
df_oot_['uw_odds_30'] = ((1 - df_oot_['gen4_underwriting_prob_30'])/df_oot_['gen4_underwriting_prob_30'])

df_oot_['gen4_underwriting_score_30'] = round(offset+factor*df_oot_['uw_log_odds_30'])
df_oot_.loc[df_oot_.gen4_underwriting_score_30>850,'gen4_underwriting_score_30']=850
df_oot_.loc[df_oot_.gen4_underwriting_score_30<300,'gen4_underwriting_score_30']=300

df_oot_['pr_log_odds'] = np.log((1 - df_oot_['gen4_prescreen_prob'])/df_oot_['gen4_prescreen_prob'])
df_oot_['pr_odds'] = ((1 - df_oot_['gen4_prescreen_prob'])/df_oot_['gen4_prescreen_prob'])

df_oot_['gen4_prescreen_score'] = round(offset+factor*df_oot_['pr_log_odds'])
df_oot_.loc[df_oot_.gen4_prescreen_score>850,'gen4_prescreen_score']=850
df_oot_.loc[df_oot_.gen4_prescreen_score<300,'gen4_prescreen_score']=300

In [15]:
df_oot_.head()

,id,applicant_type,date_start,seg,ri_source,target_v2,weight_eval,period,gen3_score,applicant_fico_score,...,gen4_underwriting_score,uw_log_odds_29,uw_odds_29,gen4_underwriting_score_29,uw_log_odds_30,uw_odds_30,gen4_underwriting_score_30,pr_log_odds,pr_odds,gen4_prescreen_score
0,6038637,PRIMARY,2018-03-19,member,others,0.0,0.997245,oot18q1,743.0,785.0,...,741.0,3.183949,24.141899,741.0,3.183949,24.141899,741.0,2.698893,14.863271,713.0
1,6038637,PRIMARY,2018-03-19,member,others,1.0,0.002755,oot18q1,743.0,785.0,...,741.0,3.183949,24.141899,741.0,3.183949,24.141899,741.0,2.698893,14.863271,713.0
2,5961856,PRIMARY,2018-03-07,member,proxy,1.0,1.000000,oot18q1,655.0,689.0,...,685.0,2.211693,9.131166,685.0,2.211693,9.131166,685.0,1.956205,7.072434,670.0
3,6078544,PRIMARY,2018-03-24,member,others,0.0,0.999152,oot18q1,698.0,744.0,...,666.0,1.886536,6.596480,666.0,1.886536,6.596480,666.0,2.738470,15.463301,715.0
4,6078544,PRIMARY,2018-03-24,member,others,1.0,0.000848,oot18q1,698.0,744.0,...,666.0,1.886536,6.596480,666.0,1.886536,6.596480,666.0,2.738470,15.463301,715.0


### save data for performance evaluation

In [16]:
df_oot_['fico_adj'] = np.where(df_oot_.applicant_fico_score>850, 300, df_oot_.applicant_fico_score)
keep_list = ['id','applicant_type','date_start','seg','ri_source','target_v2','weight_eval','gen4_prescreen_score','gen4_prescreen_prob',
             'period', 'gen4_underwriting_prob', 'gen4_underwriting_score', 'gen4_underwriting_prob_29', 'gen4_underwriting_score_29',
             'gen4_underwriting_prob_30', 'gen4_underwriting_score_30','gen3_score','fico_adj',
             'applicant_vantage_score','funds_use_encode_v2','income_update']

In [17]:
df_oot_[keep_list].to_parquet(artifact_path+ "/" + 'df_gen4_score_oot_refit.parquet')
df_oot_[keep_list].to_parquet('./artifact/df_gen4_score_oot_refit.parquet')